In [1]:
import pandas as pd 
import numpy as np
import time
import traceback

from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as sch
import scipy.spatial.distance as dist

import AllSample as ass
import Cluster as clu
import Filter as fil
import WordCount as woc
import WordCount2 as woc2
import SentimentAnalysis as sap

from flask import Flask, render_template, request, jsonify#, Response, make_response

In [2]:
app = Flask(__name__)

df = pd.DataFrame(np.random.randint(0,10,size=(10, 2)), columns=list('AB'))
#print(df)
#print(df.to_json(orient='records'))

@app.route("/")
def index():
    #print(jsonify(df.to_json(orient='records')))
    return render_template("/index3.html")

@app.route("/something", methods=['GET', 'POST'])
def something():    
    if request.method == 'POST':
        try:
            neshto = request.form['neshto']        
            status = neshto + " Bogy"
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return jsonify(df.to_json(orient='records'))
        #return "Status: " + str(status) + " \n clusNum: " + str(clusNum) + " \n dateLow: "+ str(dateLow)+ " \n dateUp: "+ str(dateUp)+ " \n toYear: "+ str(toYear)+ " \n weekDay: "+ str(weekDay)+ " \n clMethod: "+ str(clMethod)
    else:
        return "Send a proper POST request!"

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2020 11:27:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 11:27:56] "GET /static/stylesheets/index.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 11:27:56] "GET /static/stylesheets/PlovdivSans.otf HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 11:28:04] "GET / HTTP/1.1" 200 -


In [2]:
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("/index.html")
    
@app.route("/AllSample")
def allSample():
    try:
        result = ass.main()
        result.to_csv('static/stackedData.csv')
    except:
        return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
    return render_template("AllSample.html")
           
@app.route("/SingleRestaurant")
def singleRestaurant():
    return render_template("SingleRestaurant.html")       

@app.route("/Clustering")
def clustering():
    return render_template("Clustering.html")
  
@app.route("/endpointCl", methods=['GET', 'POST'])
def endpointCl():    
    if request.method == 'POST':
        try:
            clusNum = request.form['clusNum']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            toYear = request.form['toYear']
            weekDay = request.form['weekDay']
            clMethod = request.form['clMethod']
            clMetric = request.form['clMetric']
            trend = request.form['trend']
            average = request.form['average']
            normalise = request.form['normalise']
            smooth = request.form['smooth']
            filtered = fil.main("", "", dateLow, dateUp, toYear, weekDay, "")
            if (filtered.empty):
                return "No data for those filters."
        
            status = clu.main(filtered, clusNum, clMethod, clMetric, trend, average, normalise, smooth)
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return str(status)
        #return "Status: " + str(status) + " \n clusNum: " + str(clusNum) + " \n dateLow: "+ str(dateLow)+ " \n dateUp: "+ str(dateUp)+ " \n toYear: "+ str(toYear)+ " \n weekDay: "+ str(weekDay)+ " \n clMethod: "+ str(clMethod)
    else:
        return "Send a proper POST request!"
    
@app.route("/WordCount")
def wordCount():
    return render_template("WordCount.html")
    
@app.route("/endpointWC", methods=['GET', 'POST'])
def endpointWC():    
    if request.method == 'POST':
        try:
            ID = request.form['ID']        
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            toYear = request.form['toYear']
            weekDay = request.form['weekDay']
            overall = request.form['overall']
            topWords = request.form['topWords']
            granulation = request.form['granulation']
            getLemma = request.form['getLemma']
            frequency = request.form['frequency'] 
            past = request.form['past'] 
            polarity = request.form['polarity']       
            filtered = fil.main(ID, rating, dateLow, dateUp, toYear, weekDay, "")
            if (filtered.empty):
                return "No data for those filters."
            if (polarity!=""):
                filtered = sap.main(filtered)[int(polarity)]
            if (filtered.empty):
                return "No data for this polarity."
            result = woc2.main(filtered, overall, topWords, granulation, getLemma, frequency, past)
            result.to_csv('static/wordCount.csv')
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return "True"
        #return "Status: " + "True" + " \n ID: " + str(ID) + " \n rating: "+ str(rating) + " \n dateLow: "+ str(dateLow)+ " \n dateUp: "+ str(dateUp)+ " \n toYear: "+ str(toYear)+ " \n weekDay: "+ str(weekDay) + " \n overall: "+ str(overall) + " \n granulation: "+ str(granulation)
    else:
        return "Send a proper POST request!"      
    
@app.route("/endpointWC2", methods=['GET', 'POST'])
def endpointWC2():    
    if request.method == 'POST':
        try:
            ID = request.form['ID']        
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            toYear = request.form['toYear']
            weekDay = request.form['weekDay']
            overall = request.form['overall']
            topWords = request.form['topWords']
            granulation = request.form['granulation']
            getLemma = request.form['getLemma']
            frequency = request.form['frequency'] 
            polarity = request.form['polarity']   
            phrases = request.form['phrases']        
            filtered = fil.main(ID, rating, dateLow, dateUp, toYear, weekDay, "")
            if (filtered.empty):
                return "No data for those filters."
            if (polarity!=""):
                filtered = sap.main(filtered)[int(polarity)]
            if (filtered.empty):
                return "No data for this polarity."
            result = woc.main(filtered, overall, topWords, granulation, getLemma, frequency, phrases)
            result.to_csv('static/wordCount.csv')
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return "True"
        #return "Status: " + "True" + " \n ID: " + str(ID) + " \n rating: "+ str(rating) + " \n dateLow: "+ str(dateLow)+ " \n dateUp: "+ str(dateUp)+ " \n toYear: "+ str(toYear)+ " \n weekDay: "+ str(weekDay) + " \n overall: "+ str(overall) + " \n granulation: "+ str(granulation)
    else:
        return "Send a proper POST request!" 

@app.route("/epWCperCl", methods=['GET', 'POST'])
def epWCperCl():    
    if request.method == 'POST': 
        try:     
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            toYear = request.form['toYear']
            weekDay = request.form['weekDay']
            overall = request.form['overall']
            whichClu = request.form['whichClu']
            topWords = request.form['topWords']
            granulation = request.form['granulation']
            getLemma = request.form['getLemma']
            frequency = request.form['frequency']
            polarity = request.form['polarity']
            phrases = request.form['phrases']
            filtered = fil.main("", "", dateLow, dateUp, toYear, weekDay, whichClu)
            if (filtered.empty):
                return "No data for those filters."
            if (polarity!=""):
                filtered = sap.main(filtered)[int(polarity)]
            if (filtered.empty):
                return "No data for this polarity."
            result = woc.main(filtered, overall, topWords, granulation, getLemma, frequency, phrases)
            result.to_csv('static/wordCount.csv')
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return "True"
        #return "Status: " + "True" + " \n dateLow: "+ str(dateLow)+ " \n dateUp: "+ str(dateUp)+ " \n toYear: "+ str(toYear)+ " \n weekDay: "+ str(weekDay) + " \n overall: "+ str(overall)
    else:
        return "Send a proper POST request!"
    
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2020 11:32:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 11:32:14] "GET /SingleRestaurant HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 11:32:14] "GET /static/revSam.csv HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:38:10] "GET /Clustering HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:38:19] "GET /Clustering HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:38:25] "POST /endpointCl HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:38:40] "POST /endpointCl HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:39:05] "GET /SingleRestaurant HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2020 15:39:05] "GET /static/revSam.csv HTTP/1.1" 200 -


In [4]:
filtered = fil.main("", "4", "2012-1-1", "2016-12-31", "0", "", "")
start = time.time()
#result = woc.main(filtered, "False", "10", "Year", )
#result
end = time.time()
print("Svurshi " + str(end - start))

Svurshi 0.0


In [4]:
woc.main(fil.main("cYwJA2A6I12KNkm2rtXd5g" , "5", "2008-1-1", "2016-12-31", "1", "6", "")).to_csv('static/wordCount.csv')

In [2]:
filtered = fil.main('', '', "01.01.2010", "31.12.2016", "", "", "")

In [12]:
proba1 = filtered.head(1000).copy()
proba1

,business_id,stars,date,text,name,postal_code,ccur
index,,,,,,,
444610,4JNXUYY8wbaaDmk3BPzlWw,4,2015-07-25,Located next to the Eiffel tower and across th...,"""Mon Ami Gabi""",89109,7328
439720,4JNXUYY8wbaaDmk3BPzlWw,5,2015-09-13,My wife made reservations for a party of 10. W...,"""Mon Ami Gabi""",89109,7328
439719,4JNXUYY8wbaaDmk3BPzlWw,5,2013-09-22,This place has over 3000+ yelp reviews so we d...,"""Mon Ami Gabi""",89109,7328
439718,4JNXUYY8wbaaDmk3BPzlWw,1,2015-01-21,We made a reservation for 845 we show up at 84...,"""Mon Ami Gabi""",89109,7328
439716,4JNXUYY8wbaaDmk3BPzlWw,5,2013-06-01,OH MY GOD. \r\r\r\nThis place was so delicious...,"""Mon Ami Gabi""",89109,7328
...,...,...,...,...,...,...,...
444856,4JNXUYY8wbaaDmk3BPzlWw,5,2016-04-16,This is my 2nd time visiting Mon Ami Gabi and ...,"""Mon Ami Gabi""",89109,7328
444855,4JNXUYY8wbaaDmk3BPzlWw,4,2011-01-05,"From Wikipedia, ""Snail is a common name for al...","""Mon Ami Gabi""",89109,7328
444841,4JNXUYY8wbaaDmk3BPzlWw,4,2011-01-07,"As a belated birthday present, I wanted to tak...","""Mon Ami Gabi""",89109,7328


In [22]:
start = time.time()
result = woc.main(proba1, "false", "10", "Year", "true", "false", "false")
end = time.time()
print("Svurshi " + str(end - start))
result

Svurshi 0.9325449466705322


,Words: Overall,Count: Overall,Words: 2010,Count: 2010,Words: 2011,Count: 2011,Words: 2012,Count: 2012,Words: 2013,Count: 2013,Words: 2014,Count: 2014,Words: 2015,Count: 2015,Words: 2016,Count: 2016
Place,,,,,,,,,,,,,,,,
1,good,924,french,36,french,71,french,53,vegas,84,time,94,restaurant,83,restaurant,72
2,food,712,like,35,restaurant,71,time,48,french,74,vegas,90,make,73,time,68
3,great,681,time,34,like,64,like,48,patio,72,view,78,time,73,well,62
4,get,620,view,33,vegas,61,vegas,47,well,71,patio,75,us,72,vegas,59
5,order,544,bellagio,33,much,59,much,45,much,62,french,72,vegas,71,us,59
6,place,536,patio,32,patio,58,really,44,restaurant,61,well,69,delicious,67,breakfast,58
7,steak,529,restaurant,32,make,56,make,44,make,59,table,67,seat,65,patio,57
8,service,526,gabi,31,one,56,love,40,time,59,strip,65,would,62,like,54
9,come,472,cheese,30,time,56,restaurant,39,really,58,bellagio,64,view,61,nice,54


In [29]:
try:
    status = clu.main2(proba1, "6", "ward", "correlation")
except:
    print(traceback.format_exc().splitlines()[-1])

ValueError: Method 'ward' requires the distance metric to be Euclidean


In [ ]:
ass.main().sort_index()
#data = readCSV('static/revSam.csv')
#data["newID"] = data["name"].map(str) +" - "+ data["postal_code"].map(str)

#data.drop(data.columns[[0,2,3,4,5,6,7,8,9,10]],axis=1,inplace=True)
#data

In [8]:
def readCSV(item):
    data = pd.read_csv(item, dtype={0: int}, index_col=0, encoding='latin-1')
    df = pd.DataFrame(data)
    return df